In [27]:
import io
import json
import time
import unicodedata
from pathlib import Path

import folium
import httpx
import pandas as pd
import tomli
from cachetools import TTLCache
from folium import plugins
from IPython.core.display import Image
from loguru import logger
from PIL import Image as pImage

## Read API key from `.toml` file

In [20]:
def read_config_file(config_file):
    config_path = Path(config_file)
    if not config_path.exists():
        logger.error(f"Config file does not exist: {config_file}")
        raise FileNotFoundError(f"Config file does not exist: {config_file}")
    
    try:
        with config_path.open("rb") as f:
            app_config = tomli.load(f)
            n_items = len(app_config.get('NSW_OPEN_DATA', {}))
            logger.info(f"Loaded {n_items} items from {config_file}")
            return app_config
    except Exception as e:
        logger.error(f"Failed to read or parse the config file: {config_file}. Error: {e}")
        raise

In [30]:
API_CONFIG_FILE = Path('../nsw_open_data.toml')
api_config = read_config_file(API_CONFIG_FILE)

2024-03-27 12:59:11.485 | INFO     | __main__:read_config_file:11 - Loaded 5 items from ../nsw_open_data.toml


In [31]:
def show_api_key(api_config):
    try:
        api_details = api_config["NSW_OPEN_DATA"]
        api_key = api_details["API_KEY"]
        show_key = api_details.get("SHOW_KEY", False)
        url = api_details["URL"]
        obscure_length = 5
        _REPL_CHAR = "█"  # Using a direct character for simplicity

        logger.info(f"API key for: {url}")
        len_api_key = len(api_key)
        logger.info(f"{len_api_key} characters in length")

        if show_key:
            display_key = api_key
        else:
            obscured_portion = _REPL_CHAR * (len_api_key - obscure_length * 2)
            display_key = f"{api_key[:obscure_length]}{obscured_portion}{api_key[-obscure_length:]}"

        logger.info(display_key)
    except KeyError as e:
        logger.error(f"Missing key in API configuration: {e}")
    except Exception as e:
        logger.error(f"Error displaying API key: {e}")

In [32]:
show_api_key(api_config)

2024-03-27 12:59:16.997 | INFO     | __main__:show_api_key:10 - API key for: https://opendata.transport.nsw.gov.au/
2024-03-27 12:59:16.998 | INFO     | __main__:show_api_key:12 - 36 characters in length
2024-03-27 12:59:16.999 | INFO     | __main__:show_api_key:20 - fwyH5██████████████████████████mWeUk


## Example 1 - Look up cark park information

API: [https://opendata.transport.nsw.gov.au/dataset/car-park-api](https://opendata.transport.nsw.gov.au/dataset/car-park-api)

In [6]:
def get_carpark_info():
    API_URL = "https://api.transport.nsw.gov.au/v1/carpark"
    response = httpx.get(
        API_URL,
        headers={
            "Authorization": "apikey " + api_config["NSW_OPEN_DATA"]["API_KEY"],
            "Accept": "application/json",
        },
    )
    return pd.DataFrame.from_dict(response.json(), orient='index', columns=['Car Park Name'])

In [7]:
get_carpark_info()

,Car Park Name
1,Tallawong Station Car Park (historical only)
10,Warriewood Car Park
11,Narrabeen Car Park
12,Mona Vale Car Park
13,Dee Why Car Park
14,West Ryde Car Park
15,Sutherland East Parade Car Park
16,Leppington Car Park
17,Edmondson Park South Car Park
18,St Marys Car Park


## Example 2 - Get toll/motorway information

API: [https://opendata.transport.nsw.gov.au/dataset/toll-calculator-api](https://opendata.transport.nsw.gov.au/dataset/toll-calculator-api)

### Part A - Get motorway information

In [8]:
def get_motorway_info():
    API_URL = "https://api.transport.nsw.gov.au/v2/roads/toll_calc/data"
    response = httpx.get(
        API_URL,
        headers={
            "Authorization": "apikey " + api_config["NSW_OPEN_DATA"]["API_KEY"],
            "Accept": "application/json",
        },
    )
    return pd.DataFrame(response.json()["motorways"])

In [9]:
get_motorway_info()

,motorwayRef,motorwayName,motorwayProvider,chargeType,gantries
0,NCX,NorthConnex,Transurban Linkt,Fixed,"[{'gantryRef': 'NXTP1', 'gantryName': 'NXTP1 (..."
1,LCT,Lane Cove Tunnel,Transurban Linkt,Fixed,"[{'gantryRef': 'FLW', 'gantryName': 'FLW - Fal..."
2,CCT,Cross City Tunnel,Transurban Linkt,Fixed,"[{'gantryRef': 'CCT-001/2', 'gantryName': 'CCT..."
3,ED,Eastern Distributor,Transurban Linkt,Fixed,"[{'gantryRef': 'ED-A/B', 'gantryName': 'ED-A/B..."
4,M2,Hills M2,Transurban Linkt,Fixed,"[{'gantryRef': 'M2-1A/2B/1C', 'gantryName': 'M..."
5,M5,M5 South-West,Transurban Linkt,Fixed,"[{'gantryRef': 'M5-HV1-12', 'gantryName': 'M5-..."
6,M7,M7,Transurban Linkt,Distance,"[{'gantryRef': 'M7-1A/1B', 'gantryName': 'M7-1..."
7,SHB,Sydney Harbour Bridge,E-Toll,TimeofDay,"[{'gantryRef': 'SHMILS', 'gantryName': 'SHMILS..."
8,SHT,Sydney Harbour Tunnel,E-Toll,TimeofDay,"[{'gantryRef': 'SHTUN', 'gantryName': 'SHTUN -..."
9,WCX,"WestConnex (M4, M5, M8)",Transurban Linkt,Distance,"[{'gantryRef': 'TP01B', 'gantryName': 'M4 Main..."


### Part B - Toll calculation for route between two GPS positions

In [10]:
def get_toll_route():
    API_URL = "https://api.transport.nsw.gov.au/v2/roads/toll_calc/route"
    response = httpx.post(
        API_URL,
        headers={
            "Authorization": "apikey " + api_config["NSW_OPEN_DATA"]["API_KEY"],
            "Accept": "application/json",
            "Content-Type": "application/json"
        }, 
        data=data_json
    )
    return response.json()


In [43]:
data = {
    "origin": {"lat": -33.8819, "lng": 151.2517, "name": "string"},
    "destination": {"lat": -33.8509, "lng": 151.2207, "name": "string"},
    "vehicleClass": "A",
    "vehicleClassByMotorway": {
        "LCT": "A",
        "CCT": "A",
        "ED": "A",
        "M2": "A",
        "M5": "A",
        "M7": "A",
        "SHB": "A",
        "SHT": "A",
        "M4": "A",
    },
    "excludeToll": "false",
    "includeSteps": "false",
    "departureTime": "2021-08-07T05:21:12.342Z",
}

data_json = json.dumps(data).replace('"true"', 'true').replace('"false"', 'false')   # fix bool types

In [12]:
get_toll_route();

## Generalisation



In [33]:


# Initialize a cache with a maximum size of 100 items and items expire after 3600 seconds (1 hour)
cache = TTLCache(maxsize=100, ttl=3600)

def make_api_request(api_name, endpoint, method="GET", headers=None, data=None):
    # Retrieve API configuration
    base_url = api_config[api_name]["BASE_URL"]
    common_headers = api_config[api_name]["HEADERS"]
    
    # Construct the full URL
    url = f"{base_url}{endpoint}"
    
    # Merge common headers with any additional headers
    if headers:
        all_headers = {**common_headers, **headers}
    else:
        all_headers = common_headers
    
    # Generate a unique cache key based on the request parameters
    cache_key = (url, method, frozenset(all_headers.items()), frozenset(data.items()) if data else None)
    
    # Check if the response is cached
    if cache_key in cache:
        return cache[cache_key]
    
    # Make the API request
    try:
        if method.upper() == "GET":
            response = httpx.get(url, headers=all_headers)
        elif method.upper() == "POST":
            response = httpx.post(url, headers=all_headers, json=data)
        else:
            raise ValueError("Unsupported HTTP method")
        
        response.raise_for_status()  # Raise an exception for HTTP error responses
        result = response.json()
        
        # Cache the result
        cache[cache_key] = result
        
        return result
    except Exception as e:
        logger.error(f"Error making API request: {e}")
        return None



In [36]:
def get_carpark_info_new():
    endpoint = "/v1/carpark"
    response = make_api_request("NSW_OPEN_DATA", endpoint)
    if response:
        return pd.DataFrame.from_dict(response, orient='index', columns=['Car Park Name'])
    else:
        return pd.DataFrame()  # Return an empty DataFrame in case of an error

In [37]:
get_carpark_info_new()

,Car Park Name
1,Tallawong Station Car Park (historical only)
10,Warriewood Car Park
11,Narrabeen Car Park
12,Mona Vale Car Park
13,Dee Why Car Park
14,West Ryde Car Park
15,Sutherland East Parade Car Park
16,Leppington Car Park
17,Edmondson Park South Car Park
18,St Marys Car Park


In [38]:
def get_motorway_info_new():
    endpoint = "/v2/roads/toll_calc/data"
    response = make_api_request("NSW_OPEN_DATA", endpoint)
    if response:
        return pd.DataFrame(response["motorways"])
    else:
        return pd.DataFrame()  # Return an empty DataFrame in case of an error

In [40]:
get_motorway_info_new()

,motorwayRef,motorwayName,motorwayProvider,chargeType,gantries
0,NCX,NorthConnex,Transurban Linkt,Fixed,"[{'gantryRef': 'NXTP1', 'gantryName': 'NXTP1 (..."
1,LCT,Lane Cove Tunnel,Transurban Linkt,Fixed,"[{'gantryRef': 'FLW', 'gantryName': 'FLW - Fal..."
2,CCT,Cross City Tunnel,Transurban Linkt,Fixed,"[{'gantryRef': 'CCT-001/2', 'gantryName': 'CCT..."
3,ED,Eastern Distributor,Transurban Linkt,Fixed,"[{'gantryRef': 'ED-A/B', 'gantryName': 'ED-A/B..."
4,M2,Hills M2,Transurban Linkt,Fixed,"[{'gantryRef': 'M2-1A/2B/1C', 'gantryName': 'M..."
5,M5,M5 South-West,Transurban Linkt,Fixed,"[{'gantryRef': 'M5-HV1-12', 'gantryName': 'M5-..."
6,M7,M7,Transurban Linkt,Distance,"[{'gantryRef': 'M7-1A/1B', 'gantryName': 'M7-1..."
7,SHB,Sydney Harbour Bridge,E-Toll,TimeofDay,"[{'gantryRef': 'SHMILS', 'gantryName': 'SHMILS..."
8,SHT,Sydney Harbour Tunnel,E-Toll,TimeofDay,"[{'gantryRef': 'SHTUN', 'gantryName': 'SHTUN -..."
9,WCX,"WestConnex (M4, M5, M8)",Transurban Linkt,Distance,"[{'gantryRef': 'TP01B', 'gantryName': 'M4 Main..."


In [42]:
def get_toll_route_new(data):
    endpoint = "/v2/roads/toll_calc/route"
    headers = {"Content-Type": "application/json"}  # Additional headers specific to this request
    response = make_api_request("NSW_OPEN_DATA", endpoint, method="POST", headers=headers, data=data)
    return response

In [46]:
get_toll_route_new(data)

TypeError: unhashable type: 'dict'

## Example 3 - Lookup address

API: [https://data.nsw.gov.au/data/dataset/lpi-web-services-address-location-service](https://data.nsw.gov.au/data/dataset/lpi-web-services-address-location-service)

In [13]:
## Address Lookup (legacy API example)

def get_address_info(houseNumber, roadName, roadType, suburb, postCode):
    URL = "http://maps.six.nsw.gov.au/services/public/Address_Location?"
    URL += "houseNumber=" + houseNumber
    URL += "&roadName=" + roadName
    URL += "&roadType=" + roadType
    URL += "&suburb=" + suburb
    URL += "&postCode" + postCode
    URL += "&projection=EPSG%3A4326"
    # logger.info(URL)
    response = httpx.get(URL)
    return response.json()['addressResult']['addresses']

In [14]:
# Example - Kirribilli House
get_address_info('109', 'Kirribilli', 'Ave', 'Kirribilli', '2061')

[{'objectId': 5894870,
  'propid': 3589465,
  'houseNumberString': '109',
  'houseNumberFirst': 109,
  'houseNumberSecond': 0,
  'roadName': 'Kirribilli',
  'roadType': 'Avenue',
  'suburbName': 'Kirribilli',
  'postCode': 2061,
  'council': 'North Sydney',
  'addressType': 'Assigned',
  'shortAddressString': '109 KIRRIBILLI AVENUE KIRRIBILLI',
  'addressPoint': {'addressstringOid': 0,
   'coordRefSys': 'EPSG:4283',
   'centreX': 151.2186838704801,
   'centreY': -33.85184922241778},
  'addressString': '109  Kirribilli Avenue  Kirribilli 2061'}]

In [15]:
def map_address(houseNumber, roadName, roadType, suburb, postCode):
    longitude = get_address_info(houseNumber, roadName, roadType, suburb, postCode)[0]['addressPoint']['centreX']
    latitude = get_address_info(houseNumber, roadName, roadType, suburb, postCode)[0]['addressPoint']['centreY']
    addressString = get_address_info(houseNumber, roadName, roadType, suburb, postCode)[0]['addressString']
    fmap = folium.Map(location=[latitude, longitude], zoom_start=13)
    tooltip = "Click me!"
    folium.Marker([latitude, longitude], popup=addressString, tooltip=tooltip).add_to(fmap)
    return fmap


In [16]:
mini_map = plugins.MiniMap(toggle_display=True)
fmap = map_address('109', 'Kirribilli', 'Ave', 'Kirribilli', '2061')
fmap.add_child(mini_map);

In [17]:
def display_and_export_map(png_filename="mymap.png"):
    image_data = fmap._to_png()
    image = pImage.open(io.BytesIO(image_data))
    #image.show()
    image.save(png_filename)
    #display(Image(image_data))
    return png_filename

In [19]:
display_and_export_map()


'mymap.png'

![My Map](mymap.png)